In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies = pd.read_csv(r'Datasets\transformed_movies.csv')
credits = pd.read_csv(r'Datasets\reduced_credits.csv')

In [3]:
# Crear una representación binaria de los géneros
mlb = MultiLabelBinarizer()
genres_matrix = mlb.fit_transform(movies['genres_encoded'])

# Combinar con otras características como popularidad
movies_features = pd.DataFrame(genres_matrix, columns=mlb.classes_)
movies_features['popularity'] = movies['popularity']

# Rellenar los valores nulos en la columna de colecciones con 'No Collection'
movies['collection'] = movies['collection'].fillna('No Collection')

# Usar OneHotEncoder para las colecciones
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
collection_encoded = ohe.fit_transform(movies[['collection']])

# Crear un DataFrame con las colecciones codificadas
collection_df = pd.DataFrame(collection_encoded, columns=ohe.get_feature_names_out(['collection']))

# Combinar con la matriz de características existente
movies_features = pd.concat([movies_features, collection_df], axis=1)

# Escalar todas las características para que las colecciones no dominen
scaler = StandardScaler()
movies_features_scaled = scaler.fit_transform(movies_features)

# Crear un nuevo DataFrame escalado
movies_features_scaled = pd.DataFrame(movies_features_scaled, columns=movies_features.columns)

# Combinar todas las características en una sola matriz: combina géneros, popularidad y colecciones (ya procesadas con OneHotEncoding)
combined_features = np.hstack([movies_features.values, collection_encoded])

# Guardar la matriz en un archivo .npy
np.save('Datasets/combined_features.npy', combined_features)
